<a href="https://colab.research.google.com/github/Raian-Rahman/Data-Mining-Implementation/blob/main/Apriori_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
In this notebook, we will implement apriori algorithm for frequent itemset mining. It is a association rule mining algorithm. In general, there are two steps in apriori algorithm. These are:
- Joining
- Pruning

The code is implemented here in python

### Loading the libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [256]:
import numpy as np
import pandas as pd
from itertools import permutations

### Loading the dataset

The dataset contains around 57000 unique transactions. So we are only taking first 1000 for our task. 

In [303]:
!gdown --id '1XmalzSRKv-J36bvV88Mi3K6KYY_cwpxP'

Downloading...
From: https://drive.google.com/uc?id=1XmalzSRKv-J36bvV88Mi3K6KYY_cwpxP
To: /content/bookstore_transactions.csv
100% 2.06k/2.06k [00:00<00:00, 4.70MB/s]


In [324]:
df = pd.read_csv('/content/drive/MyDrive/Data Mining Assignment/shortened_dataset.csv')
# df = df[:1000]
df.head()

,Unnamed: 0,order_id,products
0,0,1,"['citrus fruit', 'semi-finished bread', 'marga..."
1,1,2,"['tropical fruit', 'yogurt', 'coffee']"
2,2,3,['whole milk']
3,3,4,"['pip fruit', 'yogurt', 'cream cheese', 'meat ..."
4,4,5,"['other vegetables', 'whole milk', 'condensed ..."


In [325]:
item_LUT = dict()
item_index = 1
transaction = dict()
for index in df.index:
    product_id = df['products'][index]
    product_id = product_id.strip('][').split(', ')
    # print(product_id)
    
    item_list = list()
    for item in product_id: 
        if item_LUT.get(item,-100) == -100:
            #its not on the list
            item_LUT[item] = item_index
            item_index+=1

        item_list.append(item_LUT[item])
    transaction[item_index] = item_list
    item_index+=1

In [326]:
transaction

{5: [1, 2, 3, 4],
 9: [6, 7, 8],
 11: [10],
 15: [12, 7, 13, 14],
 19: [16, 10, 17, 18],
 23: [10, 20, 7, 21, 22],
 25: [24],
 29: [16, 26, 24, 27, 28],
 31: [30],
 33: [10, 32],
 37: [6, 16, 34, 35, 36],
 41: [1, 6, 10, 20, 38, 7, 39, 35, 40],
 43: [42],
 46: [44, 24, 45],
 48: [47, 6],
 52: [20, 49, 50, 51],
 53: [50],
 55: [54],
 56: [36],
 58: [57],
 59: [16],
 62: [60, 61],
 63: [10],
 66: [6, 13, 64, 65, 51],
 74: [6, 67, 16, 68, 24, 39, 69, 70, 71, 72, 73],
 76: [35, 75],
 77: [7],
 79: [78, 24, 45, 36],
 80: [16],
 83: [81, 45, 50, 75, 51, 82],
 85: [7, 84, 35, 57],
 90: [86, 16, 24, 87, 35, 88, 89],
 91: [67, 16, 10, 84, 49],
 96: [92, 93, 16, 10, 94, 95, 45, 22],
 98: [42, 97, 65],
 99: [61, 45],
 100: [50],
 101: [75],
 103: [67, 16, 10, 102],
 105: [1, 104, 51],
 106: [78, 24, 45, 75, 57, 82],
 108: [6, 67, 10, 7, 107, 81, 61, 49, 32, 8, 45, 71, 72],
 109: [93, 7],
 110: [75],
 112: [60, 7, 13, 111, 24, 35, 45, 51],
 113: [8],
 114: [61, 35],
 115: [24],
 117: [116],
 119: 

In [327]:
processed_transaction = []
for key in transaction.keys():
    row = dict()
    row['order_id'] = key
    row['products'] = transaction[key]
    processed_transaction.append(row)

In [328]:
processed_transaction

[{'order_id': 5, 'products': [1, 2, 3, 4]},
 {'order_id': 9, 'products': [6, 7, 8]},
 {'order_id': 11, 'products': [10]},
 {'order_id': 15, 'products': [12, 7, 13, 14]},
 {'order_id': 19, 'products': [16, 10, 17, 18]},
 {'order_id': 23, 'products': [10, 20, 7, 21, 22]},
 {'order_id': 25, 'products': [24]},
 {'order_id': 29, 'products': [16, 26, 24, 27, 28]},
 {'order_id': 31, 'products': [30]},
 {'order_id': 33, 'products': [10, 32]},
 {'order_id': 37, 'products': [6, 16, 34, 35, 36]},
 {'order_id': 41, 'products': [1, 6, 10, 20, 38, 7, 39, 35, 40]},
 {'order_id': 43, 'products': [42]},
 {'order_id': 46, 'products': [44, 24, 45]},
 {'order_id': 48, 'products': [47, 6]},
 {'order_id': 52, 'products': [20, 49, 50, 51]},
 {'order_id': 53, 'products': [50]},
 {'order_id': 55, 'products': [54]},
 {'order_id': 56, 'products': [36]},
 {'order_id': 58, 'products': [57]},
 {'order_id': 59, 'products': [16]},
 {'order_id': 62, 'products': [60, 61]},
 {'order_id': 63, 'products': [10]},
 {'order_

In [329]:
processed_df = pd.DataFrame.from_dict(processed_transaction)

In [330]:
processed_df.head()

,order_id,products
0,5,"[1, 2, 3, 4]"
1,9,"[6, 7, 8]"
2,11,[10]
3,15,"[12, 7, 13, 14]"
4,19,"[16, 10, 17, 18]"


In [332]:
def generate_permutations(items, lenght_of_permutations):
    item_set = list()
    unique_items = dict()
    for item in permutations(items,lenght_of_permutations):
        
        item_list = list()
        
        for product in item:
            item_list.append(product)
        
        item_list.sort()

        if unique_items.get(tuple(item_list),-100) == -100:
            item_set.append(item)
            unique_items[tuple(item_list)] = 1

    return item_set

# """
# Driver code
# # """
items = generate_permutations([1, 2, 3, 4, 5],2)
print(items)
# for item in items:
#     for prod in item:
#         print(prod)

[(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


In [333]:
def get_itemset(transaction_list, item_list: list, length_of_permutations, global_item_count):
    item_count = dict()
    
    item_set = generate_permutations(item_list, length_of_permutations)

    for items in item_set:
        item_count[items] = 0
        global_item_count[items] = 0
        for transaction in transaction_list:
            count = 0
            for item in items:
                if item in transaction:
                    count +=1
            if count == len(items):
                item_count[items] += 1
                global_item_count[items] += 1

    return item_count, global_item_count

In [334]:
def prunning(itemset_count, support_count):
    pruned_itemset = dict()
    pruned_items = list()
    # for prunning and unique item_list

    for keys in itemset_count:
        if itemset_count[keys] >= support_count :
            pruned_itemset[keys] = itemset_count[keys]
            
            for item in keys:
                pruned_items.append(item)


    pruned_items = list(set(pruned_items))
    return pruned_itemset, pruned_items

# prunning(get_itemset(transaction_list, item_list, 2),3)

In [335]:
def level_wise_search(transaction_list, support_count):
    unique_item_list = list()
    for items in transaction_list:
        for item in items:
            unique_item_list.append(item)

    unique_item_list = list(set(unique_item_list))

    length_of_permutations = 1
    
    temp_pruned_list = dict()
    global_item_count = dict()

    while(len(unique_item_list) > 0):
        print(f'calculating on level {length_of_permutations}')
        itemsets, global_item_count = get_itemset(transaction_list, unique_item_list, length_of_permutations, global_item_count)
        pruned_itemset, unique_item_list = prunning(itemsets, support_count)
        print(f'Item list after prunning: {pruned_itemset}')
        if len(unique_item_list) > 0 :
            temp_pruned_list = pruned_itemset
        
        length_of_permutations += 1
    
    print(f'Completed generating frequent itemset with support count')
    return temp_pruned_list, global_item_count    

In [336]:
from itertools import chain, combinations

def generate_subsets(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1)))

print(generate_subsets([1,2,3]))

[(1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]


In [347]:
def recommendation(frequent_itemset, min_conf_value, global_item_count):
    """
    here we will iterate over all frequent itemsets to generate recommendations
    """    
    for keys in frequent_itemset:
        item_list = list(keys)
        subsets = generate_subsets(item_list)
        # print(subsets)
        for items in subsets:
            support_i = global_item_count[keys]
            support_s = global_item_count[tuple(items)]
            # print(keys, support_i,tuple(items), support_s)
            
            if float(support_i/support_s) >= min_conf_value:
                recommended_item = list()
                for products in item_list:
                    if products not in items:
                        recommended_item.append(products)
                
                if len(recommended_item) > 0:
                    print(f'{list(items)} \t->\t {recommended_item}\t with confidence: {float(support_i/support_s)}')


In [348]:
def apriori(transaction_list, support_count=1, min_conf=0.5):
    item_set, global_item_count = level_wise_search(transaction_list, support_count)
    recommendation(item_set, min_conf, global_item_count)

In [349]:
transaction_list = processed_df['products'].to_list()
apriori(transaction_list,250,.4)

calculating on level 1
Item list after prunning: {(1,): 814, (3,): 576, (6,): 1032, (7,): 1372, (8,): 571, (10,): 2513, (12,): 744, (13,): 390, (16,): 1903, (18,): 368, (20,): 545, (24,): 1809, (26,): 329, (27,): 792, (34,): 414, (35,): 1087, (36,): 488, (38,): 524, (42,): 516, (44,): 580, (45,): 1715, (47,): 422, (49,): 333, (50,): 711, (51,): 785, (57,): 269, (60,): 275, (61,): 875, (67,): 1072, (70,): 372, (71,): 378, (72,): 294, (75,): 764, (78,): 924, (81,): 638, (82,): 969, (84,): 256, (86,): 327, (88,): 324, (89,): 515, (92,): 567, (93,): 327, (94,): 705, (102,): 365, (107,): 624, (116,): 279, (123,): 256, (132,): 276, (139,): 473, (157,): 299, (170,): 279, (175,): 305, (350,): 254}
calculating on level 2
Item list after prunning: {(1, 10): 300, (1, 16): 284, (6, 7): 288, (6, 10): 416, (6, 16): 353, (7, 10): 551, (7, 16): 427, (7, 24): 338, (7, 45): 269, (7, 67): 254, (10, 12): 296, (10, 16): 736, (10, 20): 271, (10, 24): 557, (10, 35): 338, (10, 38): 257, (10, 45): 394, (10, 50